In [1]:
%load_ext autoreload
%autoreload 2
import functions as f
from blockchain import Blockchain

Al crear una instancia, se debe determinar la cantidad máxima de transacciones por bloque, donde se tiene en cuenta a cada envio de fees y al pago al minero como transacciones.

También se asigna un número natural "pow_difficulty", que indica la cantidad de ceros iniciales que debe tener el hash del bloque para completar la prueba de trabajo (PoW).

In [2]:
bc = Blockchain(max_transactions_per_block = 5, pow_difficulty = 3)

Al inicializar la instancia, se mina el bloque cero y se crea una única cuenta llamada "miner" al que se le asigna cash luego de cada PoW.

In [3]:
bc.wallets_taking_only_mined_blocks()

{'miner': 100.0}


Luego de realizar PoW, se crea automáticamente un nuevo bloque, listo para recibir transacciones.

Con "bc.show()" se muestra todos los bloques, incluso el no minado (el último).

En el segundo bloque se le asigna menos cash al minero. A medida que el bloque es más largo, se le entrega menos premio al minero, eventualmente llegando a 0.

In [4]:
bc.show()


block  0

header:
nonce  :  4556
block_number  :  0
merkle_tree_root_hash  :  c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841
timestamp  :  1655843836602

body:
transactions  :  (('miner', 100.0),)
merkle_tree  :  Node('/c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841')
wallets_after_block_transactions  :  {'miner': 100.0}

 ----------------------------------------------------------------------------------------------------

block  1

header:
nonce  :  0
prev_hash  :  00026d2ab0a249a731c8ac37599e3746cb97f38edfbb0e22bbab8ee72b09bc2a
block_number  :  1

body:
transactions  :  (('miner', 50.0),)

 ----------------------------------------------------------------------------------------------------


Mostramos las wallets existentes con el cash que les corresponde, tomando en cuenta solamente los bloques minados:

Agregamos dos cuentas. Al inicializarlas, ambas tienen cash = 0.

Solo se puede crear cash a través del minado.

In [5]:
bc.add_account("Alice")
bc.add_account("Bob")

Agregamos dos transacciones al bloque no minado.

Las transacciones se realizan con una tupla de la forma (payer,payee,amount_to_transfer,fee).

El fee se determina al momento de realizar la transacción.

En este caso el único que tiene cash es el minero, entonces es él el que envía. También se paga fees a si mismo en este caso.

In [6]:
bc.add_transaction(("miner","Alice",10,1))

Current block transactions =  3 . Max allowed =  5


Intentamos que la cuenta "miner" de más cash del que tiene a "Alice":

In [7]:
bc.add_transaction(("miner","Alice",1000,1))

AssertionError: 

En la transacción anterior, el minero le dio cash a Alice, entonces se puede agregar una transacción en el mismo bloque con un envío de Alice hacia otro usuario:

In [8]:
bc.add_transaction(("Alice","Bob",5,1))

Current block transactions =  5 . Max allowed =  5


Si se intenta agregar más transacciones de lo permitido, se devuelve un error:

In [9]:
bc.add_transaction(("miner","Bob",20,1))

ValueError: The block is full. Mine it to add a transaction in a new block. 

In [10]:
bc.show()


block  0

header:
nonce  :  4556
block_number  :  0
merkle_tree_root_hash  :  c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841
timestamp  :  1655843836602

body:
transactions  :  (('miner', 100.0),)
merkle_tree  :  Node('/c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841')
wallets_after_block_transactions  :  {'miner': 100.0}

 ----------------------------------------------------------------------------------------------------

block  1

header:
nonce  :  0
prev_hash  :  00026d2ab0a249a731c8ac37599e3746cb97f38edfbb0e22bbab8ee72b09bc2a
block_number  :  1

body:
transactions  :  (('miner', 50.0), ('miner', 'Alice', 10), ('miner', 'miner', 1), ('Alice', 'Bob', 5), ('Alice', 'miner', 1))

 ----------------------------------------------------------------------------------------------------


Minamos el último bloque. Si se desea también se puede minarlo cuando no tiene el número máximo de transacciones.

In [11]:
bc.mine_last_block()

In [12]:
bc.show()


block  0

header:
nonce  :  4556
block_number  :  0
merkle_tree_root_hash  :  c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841
timestamp  :  1655843836602

body:
transactions  :  (('miner', 100.0),)
merkle_tree  :  Node('/c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841')
wallets_after_block_transactions  :  {'miner': 100.0}

 ----------------------------------------------------------------------------------------------------

block  1

header:
nonce  :  3600
prev_hash  :  00026d2ab0a249a731c8ac37599e3746cb97f38edfbb0e22bbab8ee72b09bc2a
block_number  :  1
merkle_tree_root_hash  :  0050379c95251cf5ea04f188a51f2950b1bddbd7b1a65dd5b942fe1262430eb1
timestamp  :  1655843966162

body:
transactions  :  (('miner', 50.0), ('miner', 'Alice', 10), ('miner', 'miner', 1), ('Alice', 'Bob', 5), ('Alice', 'miner', 1))
merkle_tree  :  Node('/0050379c95251cf5ea04f188a51f2950b1bddbd7b1a65dd5b942fe1262430eb1')
wallets_after_block_transactions  :  {'miner': 141.0, 'Alice': 

Visualizamos como quedó el Merkle Tree del último bloque minado

In [13]:
bc.visualize_merkle_tree(number_block = -2)

0050379c95251cf5ea04f188a51f2950b1bddbd7b1a65dd5b942fe1262430eb1
├── 45ec05d1f448ba5241da2402bfcd9b15f47994a45e36ad00f66550297fe2fe6e
│   └── d6fce19356c28302e073cfd0214f5e19010224d476360e693f651dfdac976dbb
│       └── bb0ee4d3786b6a31864a3dfb1d50a8f2c2f70cadd12c013a80f75c85ee8e28e7
│           └── ('miner', 50.0)
└── b5d5062ea527c44157df087935fa66e727015dacde2e4d451cf3252be4268111
    ├── c65c087034fb725599502e1c773e14cddc7c3cc73b7d82c38016ed18e2e15e1b
    │   ├── b4aabe4d189f69d4ce43528d2db383d0d95edca87fe71c05174a9abfdbcf9333
    │   │   └── ('miner', 'Alice', 10)
    │   └── 19be4cfe23fb44f0bb679d993561ff582d010775a264d53a59bdf0c42eec785d
    │       └── ('miner', 'miner', 1)
    └── 5fedca3619300f47dbd7e11622121d248936f4def2534ae6172036bc972283c7
        ├── e49c742a51b58215a3597492e160cce2df79f29443ab317628f28b2afe49c943
        │   └── ('Alice', 'Bob', 5)
        └── cd4dddb88b4513a320c3022c8fd7e4a6897e2b66c32df917762fe936fa2b98de
            └── ('Alice', 'miner', 1)


Vemos si el hash del head del último bloque minado es el hash guardado en el bloque siguiente:

In [15]:
idx = -2
f.string_to_hash(str((bc.chain[idx]["header"]))) == bc.chain[idx+1]["header"]["prev_hash"]

True

Veamos el estado de las cuentas y minemos un bloque realizando una único envío de dinero.

In [17]:
bc.wallets_taking_only_mined_blocks()

{'miner': 141.0, 'Alice': 4, 'Bob': 5}


Realizamos un envio de dinero de Bob a si mismo:

In [20]:
bc.add_transaction(("Bob","Bob",2,1))

Current block transactions =  3 . Max allowed =  5


In [21]:
bc.mine_last_block()

In [22]:
bc.show()


block  0

header:
nonce  :  4556
block_number  :  0
merkle_tree_root_hash  :  c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841
timestamp  :  1655843836602

body:
transactions  :  (('miner', 100.0),)
merkle_tree  :  Node('/c46ca60643d459f7b73c9bf6fde6ae366f8671bb10e4f3228e8edb77df3f7841')
wallets_after_block_transactions  :  {'miner': 100.0}

 ----------------------------------------------------------------------------------------------------

block  1

header:
nonce  :  3600
prev_hash  :  00026d2ab0a249a731c8ac37599e3746cb97f38edfbb0e22bbab8ee72b09bc2a
block_number  :  1
merkle_tree_root_hash  :  0050379c95251cf5ea04f188a51f2950b1bddbd7b1a65dd5b942fe1262430eb1
timestamp  :  1655843966162

body:
transactions  :  (('miner', 50.0), ('miner', 'Alice', 10), ('miner', 'miner', 1), ('Alice', 'Bob', 5), ('Alice', 'miner', 1))
merkle_tree  :  Node('/0050379c95251cf5ea04f188a51f2950b1bddbd7b1a65dd5b942fe1262430eb1')
wallets_after_block_transactions  :  {'miner': 141.0, 'Alice': 

In [23]:
bc.visualize_merkle_tree(number_block = -2)

47b4841b207a4c1448348eae42228d17bc3dd8dcf15bfbbedc88255567887cc3
├── 7921d5246cf35f6bc33376028ea662626a950852573043e674f1744a023ba079
│   └── 24ea2234316139760800fc01a2a7e33752238d28c981627aa516e1c7d6cdfc34
│       └── ('miner', 33.333)
└── a9008de30b09b893f21d2a391f47bd90767de3629c1bc83cf8a913a35c8281f9
    ├── 02afb1d376e8fe98e01d77a759c261e41051e5105eeab75aebbcceb0c1a002a0
    │   └── ('Bob', 'Bob', 2)
    └── e14b314e03bfd95c26374ac8a3837b39322c26e68ab1e72a2157506aabf6acd2
        └── ('Bob', 'miner', 1)


In [24]:
bc.wallets_taking_only_mined_blocks()

{'miner': 175.333, 'Alice': 4, 'Bob': 4}


Chequeamos si un bloque dado es mas nuevo que otro:

In [25]:
print(f.block_newer_than(bc.chain[-2],bc.chain[-3]))
print(f.block_newer_than(bc.chain[-4],bc.chain[-2]))

True
False


Intentamos realizar una transaccion usando una cuenta inexistente:

In [28]:
#error para wallet inexistente
bc.add_transaction(("Alice1","Bob",5,1))

KeyError: 'Alice1'